# Topo - VTK 4x4 transform matrix from control points
O. Kaufmann, 2021.

In [1]:
from ipywidgets import TwoByTwoLayout
import bootsoff.topo.geometry as btg
import pyvista as pv
import numpy as np

## Coordinates of the control points in the origin space 
at least 4 non co-planar points are required, if more than 4 are given, a least-squares approach is used

In [2]:
p0 = np.array([0., 0., 0.])
p1 = np.array([0., 4., 0.])
p2 = np.array([1., 0., 0.])
p3 = np.array([0., 0., 2.])

In [3]:
origin_points = [p0, p1, p2, p3]

## Corresponding coordinates of the control points in the destination space 

In [4]:
P0 = np.array([2., 0., 0.])
P1 = np.array([2., 0., 1.])
P2 = np.array([3., 0., 0.])
P3 = np.array([2., -1., 3.])

In [5]:
destination_points = [P0, P1, P2, P3]

## Convert control points to vtk objects

In [6]:
origin = pv.PolyData(origin_points)
origin['Labels'] = [f'p{i}' for i in range(4)]
destination = pv.PolyData(destination_points)
destination['Labels'] = [f'P{i}' for i in range(4)]

2021-08-07 20:21:20.950 (   0.552s) [        14862740]vtkDataSetAttributes.cx:1285  WARN| vtkPointData (0x55fda4f3a890): Can not set attribute Scalars. Only vtkDataArray subclasses can be set as active attributes.
2021-08-07 20:21:20.951 (   0.553s) [        14862740]vtkDataSetAttributes.cx:1285  WARN| vtkPointData (0x55fda509b4e0): Can not set attribute Scalars. Only vtkDataArray subclasses can be set as active attributes.


## Create plotters for the origin and destination spaces

In [7]:
origin_plotter = pv.Plotter()
origin_plotter.add_point_labels(origin, 'Labels', point_size=12, font_size=12, point_color='green')
#origin_plotter.add_points(origin, point_size=12, color='green')
origin_plotter.add_axes_at_origin(labels_off=True)
origin_plotter.camera_position = 'iso'
# origin_plotter.show_grid()
origin_scene = origin_plotter.show(jupyter_backend='ipyvtklink', return_viewer=True)

In [8]:
destination_plotter = pv.Plotter()
destination_plotter.add_point_labels(destination, 'Labels', point_size=12, font_size=12, point_color='green')
destination_plotter.add_axes_at_origin(labels_off=True)
destination_plotter.camera_position = 'iso'
# destination_plotter.show_grid()
destination_scene = destination_plotter.show(jupyter_backend='ipyvtklink', return_viewer=True)

## Compute the 4x4 transform matrix

In [9]:
transform_matrix, _ = btg.vtk_transform_matrix_from_control_points(origin_points, destination_points)

## Use the 4x4 transform to transform origin points into destination points

In [10]:
origin.clear_arrays() # must remove field before applying transform
transformed = origin.transform(transform_matrix)

In [11]:
transformed['Labels'] = [f'P{i}' for i in range(4)]

2021-08-07 20:21:21.792 (   1.394s) [        14862740]vtkDataSetAttributes.cx:1285  WARN| vtkPointData (0x55fda4f3a890): Can not set attribute Scalars. Only vtkDataArray subclasses can be set as active attributes.


In [12]:
transformed

2021-08-07 20:21:21.800 (   1.402s) [        14862740]vtkDataSetAttributes.cx:1285  WARN| vtkPointData (0x55fda4f3a890): Can not set attribute Scalars. Only vtkDataArray subclasses can be set as active attributes.


PolyData (0x7f34a3b53b80)
  N Cells:	4
  N Points:	4
  X Bounds:	2.000e+00, 3.000e+00
  Y Bounds:	-1.000e+00, 3.624e-17
  Z Bounds:	-3.886e-16, 3.000e+00
  N Arrays:	1

## Create a plotter for transformed points and display origin (top left), destination (top right) end transformed (bottom right) control points

In [13]:
transformed_plotter = pv.Plotter()
transformed_plotter.add_point_labels(transformed, 'Labels', point_size=12, font_size=12, point_color='green')
#transformed_plotter.add_points(transformed, point_size=12, color='green')
transformed_plotter.add_axes_at_origin(labels_off=True)
transformed_plotter.camera_position = 'iso'
#transformed_plotter.show_grid()
transformed_scene = transformed_plotter.show(jupyter_backend='ipyvtklink', return_viewer=True)

In [14]:
TwoByTwoLayout(top_left=origin_scene, top_right=destination_scene, bottom_right=transformed_scene)

TwoByTwoLayout(children=(ViewInteractiveWidget(height=768, layout=Layout(grid_area='top-left', height='auto', …

In [15]:
transformed

PolyData (0x7f34a3b53b80)
  N Cells:	4
  N Points:	4
  X Bounds:	2.000e+00, 3.000e+00
  Y Bounds:	-1.000e+00, 3.624e-17
  Z Bounds:	-3.886e-16, 3.000e+00
  N Arrays:	1

In [16]:
[transformed.cell_points(i) for i in range(transformed.n_cells)]

[array([[ 2.00000000e+00, -4.24729747e-17, -3.88578059e-16]]),
 array([[ 2.0000000e+00, -2.6451758e-16,  1.0000000e+00]]),
 array([[ 3.00000000e+00,  3.62356719e-17, -2.61544505e-16]]),
 array([[ 2., -1.,  3.]])]

In [17]:
[destination.cell_points(i) for i in range(destination.n_cells)]

[array([[2., 0., 0.]]),
 array([[2., 0., 1.]]),
 array([[3., 0., 0.]]),
 array([[ 2., -1.,  3.]])]